<a href="https://colab.research.google.com/github/OlgaGeta/info_search/blob/main/HW_3_infsearch_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd '/content/drive/MyDrive/data'

/content/drive/MyDrive/data


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from transformers import BertTokenizer, BertModel

In [5]:
# предобученная модель BERT для русскоязычных текстов
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
%cd ..

/content/drive/MyDrive


In [8]:
import os
import re
import torch
from collections import defaultdict
from math import log
import pymorphy2

In [9]:
source_dir = 'data'
morph = pymorphy2.MorphAnalyzer()

In [10]:
def build_inverted_index(source_dir):
    inverted_index = defaultdict(dict)
    doc_freq = defaultdict(int)
    total_docs = 0

    # Подсчет файлов в директории
    for filename in os.listdir(source_dir):
        if filename.endswith(".txt"):
            total_docs += 1
            file_path = os.path.join(source_dir, filename)

            #Разделение текста на части длиной до 512 символов(для BERT-а) и приведение к нижнему регистру
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read().lower()
                chunks = [text[i:i+512] for i in range(0, len(text), 512)]

                '''
                Для каждой разделенной части текста производится
                токенизация, затем их индексация и создание тензора с индексами
                '''
                for chunk in chunks:
                    tokens = tokenizer.tokenize(chunk)
                    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
                    input_ids = torch.tensor([indexed_tokens])

                    #Подача тензора в модель и получение векторного представления
                    with torch.no_grad():
                        outputs = model(input_ids)
                        vector_repr = outputs.last_hidden_state[0].mean(dim=0)

                    term_freq = defaultdict(int) # Словарь для хранения частоты терминов в документе
                    #Удаление пунктуации, лемматизация
                    for token in tokens:
                        token = re.sub(r'[^\w\s]', '', token)
                        lemma = morph.parse(token)[0].normal_form
                        term_freq[lemma] += 1

                    #Для каждого терма и его частоты. Происходит добавление информации о термине в обратный индекс
                    for term, freq in term_freq.items():
                        inverted_index[term][filename] = (freq, vector_repr)
                        doc_freq[term] += 1

    idf = {term: log(total_docs / freq) for term, freq in doc_freq.items()}

    for term, doc_freqs in inverted_index.items():
        for doc, (freq, vector_repr) in doc_freqs.items():
            tfidf = freq * idf[term]
            inverted_index[term][doc] = (tfidf, vector_repr)

    return inverted_index

In [11]:
def search_inverted_index(inverted_index, words):
    search_results = defaultdict(float)

    # обход слов, которые ищем и их лемматизация
    for word in words:
        lemma = morph.parse(word)[0].normal_form

        '''
          Итерация по каждому документу, связанному со словом в обратном индексе
          tfidf - вес TF-IDF слова в документе, vector_repr - векторное представление документа
        '''
        for doc, (tfidf, vector_repr) in inverted_index.get(lemma, {}).items():
            search_results[doc] += abs(tfidf)

    return search_results

In [12]:
inverted_index = build_inverted_index(source_dir)

search_words = ["симпсон", "устарел"]
results = search_inverted_index(inverted_index, search_words)

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")


Документы с искомыми словами:
postmodern.txt: 0.6931471805599453


In [13]:
search_words = ["смена", "постмодерна"]
results = search_inverted_index(inverted_index, search_words)

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")
#если оба слова не найдены в документе, то при нахождении одного,
#файл также выводится в списке, но с более низким значением

Документы с искомыми словами:
postmodern.txt: 3.9620029377331667
metamodern.txt: 2.204145020180793
allmodern.txt: 2.204145020180793
modernart.txt: 1.9810014688665833


In [14]:
search_words = ["современное", "искусство"]
results = search_inverted_index(inverted_index, search_words)

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
modernart.txt: 4.329910633534868
metamodern.txt: 2.8258332367585934
allmodern.txt: 2.8258332367585934
postmodern.txt: 1.5040773967762742


In [15]:
search_words = ["традиционализм", "современность"]
results = search_inverted_index(inverted_index, search_words)

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
metamodern.txt: 0.28768207245178085
allmodern.txt: 0.28768207245178085
postmodern.txt: 0.28768207245178085


In [16]:
search_words = ["рефлексируя", "над", "уже", "созданным"]
results = search_inverted_index(inverted_index, search_words)

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
allmodern.txt: 0.8630462173553426
modernart.txt: 0.28768207245178085
postmodern.txt: 0.0


In [17]:
search_words = ["рефлексировать", "над", "уже", "создать"]
results = search_inverted_index(inverted_index, search_words)

if results:
    sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)
    print("Документы с искомыми словами:")
    for doc, score in sorted_results:
        print(f"{doc}: {score}")
else:
    print("Нет документов с такими словами")

Документы с искомыми словами:
allmodern.txt: 0.8630462173553426
modernart.txt: 0.28768207245178085
postmodern.txt: 0.0
